#### PAIR PROGRAMMING - ETL 5


- Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [14]:
import pandas as pd
import mysql.connector

In [15]:
df_ccaa = pd.read_pickle('../ejercicio6/datos/df_ccaa.pkl')

In [16]:
df_nacional = pd.read_pickle('../ejercicio6/datos/df_nacional.pkl')

In [17]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros que son fijos a lo largo de toda la BBDD, el nombre de la BBDD y la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña
    
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
    
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
    
    def check_comunidades(self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de ciudades de la tabla de localidades 
        query_existe_comunidad = f"""
                SELECT DISTINCT comunidad FROM comunidades
                """
        mycursor.execute(query_existe_comunidad)
        comunidades = mycursor.fetchall()
        return comunidades

    def sacar_id_comunidad(self, comunidad):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idcomunidades FROM comunidades WHERE comunidad = '{comunidad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. "
    
    def sacar_id_fecha(self,fecha):
        mydb = mysql.connector.connect(user='root', password=f'{self.contraseña}',
                                          host='127.0.0.1', database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        try:
            query_sacar_id = f"SELECT idfecha FROM fecha WHERE fecha = '{fecha}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "

In [18]:
carga = Cargar("energia", "AlumnaAdalab")

In [19]:
id_comunidad = carga.sacar_id_comunidad("Ceuta")

In [20]:
id_comunidad

'Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. '

In [21]:
# tabla fecha

for indice, fila in df_nacional.iterrows(): 
    
    query_fecha = f"""
            INSERT INTO fecha (fecha) # como la primary key la hemos puesto como autoincremental no hace falta introducirla en el insert
            VALUES ( "{fila["fecha"]}");
            """
    carga.crear_insertar_tabla(query_fecha)

In [25]:
for indice, fila in df_ccaa.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidad = f"""
                INSERT INTO comunidades (comunidad) 
                VALUES ( "{fila['comunidad']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    
    carga.crear_insertar_tabla(query_comunidad)

KeyError: 'comunidad'

In [27]:
# tabla comunidades

for indice, fila in df_ccaa.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidad = f"""
                INSERT INTO comunidades (comunidad) 
                VALUES ( "{fila['comunidad']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    comunidades = carga.check_comunidades()
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
    # 
    if len(comunidades) == 0 or fila["comunidad"] not in comunidades[0]: 
        carga.crear_insertar_tabla(query_comunidad)

    else:
        print(f"{fila['comunidad']} ya esta en nuestra BBDD")

KeyError: 'comunidad'